## Cola de mensajes

- Arquitectura tradicional:
    - Protocolo de comunicaciones asíncronas
        - Entre aplicaciones
        - Entre procesos o hilos
    - Paradigma productor/consumidor
    - Control de entrega de mensajes
    - Desventajas
        - Menor rendimiento (sistemas complejos de entrega de mensajes / mantenimiento del estado)
        - Almacenaje en RAM (no persistente y limitado)

- Arquitectura para big data:
    - Simplificación de la entrega de mensajes
    - Mayor rendimiento
    - Almacenamiento persistente
    - Ejemplos:
        - Google Cloud Pub/Sub
        - Kafka
        - Pulsar
        - Amazon Kinesis

## ¿Qué es kafka?
- Plataforma unificada de alto rendimiento y baja latencia para la gestión de streams de datos.
- Permite el transporte y entrega de información big data en tiempo real.
- Proporciona almacenamiento de la entrada.
- También proporciona un motor de procesamiento de mensajes basado en eventos, en tiempo real y con semántica de entrega exactly-once (kafka streams), aunque no se va a considerar en este curso.
![02_kafka_1]

[02_kafka_1]:images/02_kafka_1.png

## Arquitectura de Kafka

- Un cluster de Kafka se compone de varios brokers:
    - Responsables de almacenar datos
    - Normalmente cada broker se encuentra en un servidor distinto
- Necesidad de almacenamiento de metainformación:
    - Zookeeper
    - Almacenamiento clave-valor jerárquico
![02_kafka_2]    

- Estructura productor/consumidor
    - Los productores escriben datos en kafka
    - Los consumidores leen datos de kafka
    
- Información organizada en topics
    - Abstracción que representa un flujo de eventos
    - El topic se divide en un conjunto de particiones
    - Cada partición es una colección de eventos numerados y ordenados en el tiempo
    - Las particiones sirven para balancear la carga de información en los brokers
    
## Replicación en Kafka

- Fundamentalmente orientada a tolerancia a fallos
- Las particiones se multiplican en distintas réplicas
- Factor de replicación: Número de copias realizadas
- Se realiza a nivel de partición: 1 máster (denominado leader), n esclavos
- Cada réplica de una partición se almacena en un broker diferente+
- La escritura y lectura se hace a través del leader de la partición
    - La replicación no influye en el balanceo de carga (se consigue por la distribución de leaders entre los brokers)
    - Replicación asíncrona: retardo desde que se escribe en el máster hasta que llega al esclavo
        - Problema: Si el máster se cae, existe pérdida de información
        - La solución sería replicación síncrona: disminución del rendimiento
        - La aplicación de esta solución depende de cómo de crítico es perder información
![02_kafka_3]

## Lectura y escritura

- Escritura:
    - Los eventos se distribuyen entre las particiones (cada evento cae sólo en una partición)
    - Operación append-only: Se escribe el nuevo evento al final de la partición
    - El orden se mantiene sólo dentro de las particiones
    - El ordinal que corresponde a un evento dentro de una partición se denomina offset
    - Tiempo de vida configurable: desde el offset más temprano hasta el último recibido
![02_kafka_4]
![02_kafka_5]
![02_kafka_6]

- Lectura:
    - Se puede leer desde el offset más temprano disponible hasta el último recibido
    - El consumidor mantiene el último offset leído y lo actualiza
    - En cada iteración se lee desde el último offset leído hasta el último recibido
    - No se almacena un estado de mensaje (quién lo ha leído)
        - Simplifica la lógica, aumenta el rendimiento y soporta cualquier número de consumidores
        - Pero la semántica ha de ser proporcionada por el consumidor, y dificulta lógicas de entrega más complejas
    - Los consumidores se organizan en grupos
    - En cada momento, una partición únicamente puede estar siendo consumida por un consumidor del grupo
    - Orden de consumición únicamente a nivel de partición
![02_kafka_7]

## Ventajas e inconvenientes

- Lectura/escritura
    - Estrategia batch:
        - Ventajas
            - Mayor rendimiento
            - Aplicable al procesamiento de datos offline
        - Inconvenientes
            - Aumenta la latencia
            - Dificultad para implementar una estrategia basada en eventos
    - Secuencial
        - Ventajas
            - Disminuye latencia
            - Aumenta rendimiento
        - Inconvenientes
            - No aplicable a cualquier tarea
            
- Particionamiento
    - Ventajas
        - Escalabilidad linear
        - Distribución uniforme del balanceo de carga
    - Incovenientes
        - Imposibilidad de restaurar el orden de eventos de forma sencilla

- Almacenamiento en disco
    - Ventajas
        - Persistencia
        - Gran cantidad de datos almacenados
    - Inconvenientes
        - Aumenta la latencia

- Otras ventajas e inconvenientes
    - Caché de página a nivel de sistema operativo
        - Mayor efectividad de lectura secuencial
        - Los datos recientes se leen de la RAM
        - Menor latencia y mayor rendimiento
    - Compresión de datos
        - Reduce uso de red y del sistema de ficheros
        - Menor latencia y mayor rendimiento
        - Aumenta el consumo de CPU

- Kafka CLI (Command Line Interface)
    - Inicialización del servidor Zookeeper
        - Definición de características en zookeeper.properties (carpeta config)
            - Puerto (clientPort, por defecto 2181)
            - Almacenamiento del snapshot (dataDir, por defecto (/tmp/zookeeper)
        - Comando de inicialización
            - zookeeper-server-start.sh config/zookeeper.properties
    - Inicialización de uno o varios brokers:
        - Definición de características en server.properties (carpeta config)
            - Identificador (broker.id)
            - Otras características
        - Comandos de inicialización
            - kafka-server-start.sh config/server.properties
        - Consultas las opcionbes en la documentación de Apache Kafka
        - Creación/eliminación/descripción de topics
            - Se indica el cluster de brokers al que conectarse mediante la opción
                "-- bootstrap-server" (por defecto en localhost:9002)
            - Comandos y opciones
                - kafka-topics.sh --create (indicar el númnero del topic, número de particiones y factor de replicación)
                - kafka-topics.sh --describe (indicar el nombre del topic)
                - kafka-topics.sh --list
                - kafka-topics.sh --delete (indicar nombre del topic)
            - Consultar las opciones en la docu...
    - Publicación en un topic
        - Se indica el cluster de brokers al que conectarse mediante la opción "--bootstrap-server"(por defecto en localhost:9092)
        - Comando
            - kafka-console-producer.sh (indicar nombre del topic)
        - Uso interactivo a través de tuberías (pipes)
        - Consultar las opciones en la documentación de Apache Kafka
    - Consumo de un topic
        - Se indica el cluster de brokers al que conectarse mediante la opción "--bootstrap-server" (por defecto en localhost:9092)
        - Comando
            - kafka-console-consumer.sh (indicar nombre del topic)
        - Algunas opciones:
            - "--from-beggining" (se lee todo lo disponible para el topic)
            - "--max-messages" (máximo de mensajes a consumir antes de finalizar)
            - "--partition" (consumo de una sola partición)
        - Consultar las opciones en la docu...
        
            


[02_kafka_2]:images/02_kafka_2.png
[02_kafka_3]:images/02_kafka_3.png
[02_kafka_4]:images/02_kafka_4.png
[02_kafka_5]:images/02_kafka_5.png
[02_kafka_6]:images/02_kafka_6.png
[02_kafka_7]:images/02_kafka_7.png